In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(r"superstore.csv")

In [7]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,11/8/18,11/11/18,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,11/8/18,11/11/18,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,6/12/18,6/16/18,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,10/11/17,10/18/17,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,10/11/17,10/18/17,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


retaining required categorical variables and dropping others

In [8]:
column = ['Order ID', 'State','Order Date', 'Ship Date', 'Customer ID', 'Customer Name', 'Country', 'City','Postal Code','Product ID','Product Name','Discount']
df.drop(column,inplace=True, axis=1)

In [9]:
df.head()

,Row ID,Ship Mode,Segment,Region,Category,Sub-Category,Sales,Quantity,Profit
0,1,Second Class,Consumer,South,Furniture,Bookcases,261.9600,2,41.9136
1,2,Second Class,Consumer,South,Furniture,Chairs,731.9400,3,219.5820
2,3,Second Class,Corporate,West,Office Supplies,Labels,14.6200,2,6.8714
3,4,Standard Class,Consumer,South,Furniture,Tables,957.5775,5,-383.0310
4,5,Standard Class,Consumer,South,Office Supplies,Storage,22.3680,2,2.5164


in this example I'm using profit for the line thickness, so I drop other variables 

In [10]:
df1 = df.filter(['Ship Mode','Segment','Region','Category','Sub-Category','Profit'], axis=1)

In [11]:
df1.head()

,Ship Mode,Segment,Region,Category,Sub-Category,Profit
0,Second Class,Consumer,South,Furniture,Bookcases,41.9136
1,Second Class,Consumer,South,Furniture,Chairs,219.5820
2,Second Class,Corporate,West,Office Supplies,Labels,6.8714
3,Standard Class,Consumer,South,Furniture,Tables,-383.0310
4,Standard Class,Consumer,South,Office Supplies,Storage,2.5164


this step onwards we specify the source and destination nodes pairwise - in the same order you want it to appear in the multi-level sankey diagram


In [12]:
df2=df1.groupby(['Region', 'Segment'], as_index=False)['Profit'].count()

In [13]:
df2.head()

,Region,Segment,Profit
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877


In [14]:
df3=df1.groupby(['Segment', 'Category'], as_index=False)['Profit'].count()

In [15]:
df3.head()

,Segment,Category,Profit
0,Consumer,Furniture,1113
1,Consumer,Office Supplies,3127
2,Consumer,Technology,951
3,Corporate,Furniture,646
4,Corporate,Office Supplies,1820


In [16]:
df4=df1.groupby(['Category','Sub-Category'], as_index=False)['Profit'].count()

In [17]:
df4.head()

,Category,Sub-Category,Profit
0,Furniture,Bookcases,228
1,Furniture,Chairs,617
2,Furniture,Furnishings,957
3,Furniture,Tables,319
4,Office Supplies,Appliances,466


In [18]:
df5=df1.groupby(['Sub-Category','Ship Mode'], as_index=False)['Profit'].count()

In [19]:
df5.head()

,Sub-Category,Ship Mode,Profit
0,Accessories,First Class,128
1,Accessories,Same Day,41
2,Accessories,Second Class,162
3,Accessories,Standard Class,444
4,Appliances,First Class,76


edit the column headers for each subset dataframe created

In [20]:
df2.columns = ['a', 'b', 'Quantity']
df3.columns = ['a', 'b', 'Quantity']
df4.columns = ['a', 'b', 'Quantity']
df5.columns = ['a', 'b', 'Quantity']

append the dataframes one below the other in the order you want

In [21]:
df6 = df2.append(df3)

In [22]:
df7 = df6.append(df4)

In [23]:
df8 = df7.append(df5)

In [24]:
df8

,a,b,Quantity
0,Central,Consumer,1212
1,Central,Corporate,673
2,Central,Home Office,438
3,East,Consumer,1469
4,East,Corporate,877
...,...,...,...
63,Supplies,Standard Class,111
64,Tables,First Class,47
65,Tables,Same Day,21
66,Tables,Second Class,61


In [25]:
#defining sankey function 

In [26]:
import pandas as pd
import plotly.graph_objects as go

def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#FFD43B','#646464','#4B8BBE','#306998']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = labelList,
            color = colorList
        ),
        link = dict(
              source = sourceTargetDf['sourceID'],
              target = sourceTargetDf['targetID'],
              value = sourceTargetDf['count']
    ))])

    return fig



In [27]:
#here we call the previously defined sankey function and we add our dataframe columns to the arguments
fig = genSankey(df8,cat_cols=['a','b'],value_cols='Quantity',title='Sankey Diagram x Profit')
fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()


All in one






In [28]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv(r"superstore.csv")

column = ['Order ID', 'State','Order Date', 'Ship Date', 'Customer ID', 'Customer Name', 'Country', 'City','Postal Code','Product ID','Product Name','Discount']
df.drop(column,inplace=True, axis=1)

df1 = df.filter(['Ship Mode','Segment','Region','Category','Sub-Category','Profit'], axis=1)

df2=df1.groupby(['Region', 'Segment'], as_index=False)['Profit'].count()
df3=df1.groupby(['Segment', 'Category'], as_index=False)['Profit'].count()
df4=df1.groupby(['Category','Sub-Category'], as_index=False)['Profit'].count()
df5=df1.groupby(['Sub-Category','Ship Mode'], as_index=False)['Profit'].count()

df2.columns = ['a', 'b', 'Quantity']
df3.columns = ['a', 'b', 'Quantity']
df4.columns = ['a', 'b', 'Quantity']
df5.columns = ['a', 'b', 'Quantity']
df6 = df2.append(df3)
df7 = df6.append(df4)
df8 = df7.append(df5)
df8





def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#FFD43B','#646464','#4B8BBE','#306998']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = labelList,
            color = colorList
        ),
        link = dict(
              source = sourceTargetDf['sourceID'],
              target = sourceTargetDf['targetID'],
              value = sourceTargetDf['count']
    ))])

    return fig

fig = genSankey(df8,cat_cols=['a','b'],value_cols='Quantity',title='Sankey Diagram x Profit')
fig.show()


